In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score  
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

from xgboost import XGBClassifier


In [ ]:
polish_test=pd.read_csv('../data/5year-Ts.csv')

y_test=polish_test['LABEL']
x_test=polish_test.drop(['LABEL','Company'],axis=1)
x_test=x_test.replace(' ', np.nan).astype(float)
for col in x_test.columns:
    avg = x_test[col].mean()
    x_test[col].fillna(avg, inplace=True)

train_data=pd.read_csv('train_data.csv')
val_data=pd.read_csv('val_data.csv')
val_y=pd.read_csv('val_y.csv')['0'].values
train_y=pd.read_csv('train_y.csv')['0'].values
train_y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
x_train= x_scaler.fit_transform(train_data)
x_test= x_scaler.fit_transform(x_test)
x_val=x_scaler.fit_transform(val_data)
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(4018, 64)
(1182, 64)
(710, 64)


In [ ]:
model = XGBClassifier(max_depth=8)
model.fit(x_train, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
y_predict5_xgb=model.predict_proba(x_val)[:, 1]


y_predict2=model.predict_proba(x_val)[:, 1]

#find the optimal threshold
precision2, recall2, thresholds2 = precision_recall_curve(val_y, y_predict2)

df_recall_precision2 = pd.DataFrame({'Precision':precision2[:-1],
                                    'Recall':recall2[:-1],
                                    'Threshold':thresholds2})
df_recall_precision2



,Precision,Recall,Threshold
0,0.069014,1.000000,0.000289
1,0.069111,1.000000,0.000524
2,0.069209,1.000000,0.000560
3,0.069307,1.000000,0.000586
4,0.069405,1.000000,0.000649
...,...,...,...
704,0.600000,0.061224,0.993493
705,0.500000,0.040816,0.993847
706,0.666667,0.040816,0.995013
707,0.500000,0.020408,0.996201


In [ ]:
np.seterr(divide='ignore', invalid='ignore')
f1_score = (2 * precision2 * recall2) / (precision2 + recall2)
findex=list(f1_score).index(max(f1_score))
thresholdOpt = round(thresholds2[findex], ndigits = 4)
fscoreOpt = round(f1_score[findex], ndigits = 4)
recallOpt = round(recall2[findex], ndigits = 4)
precisionOpt = round(precision2[findex], ndigits = 4)
print('Best Threshold: {} , F-Score: {}'.format(thresholdOpt, fscoreOpt))
print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))

from sklearn.metrics import precision_score, classification_report, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
y_pre_test2=model.predict_proba(x_test)[:, 1]
defaulter_decision_2 = (y_pre_test2 >= thresholdOpt)
print(classification_report(y_test, defaulter_decision_2))



Best Threshold: 0.9434000253677368 , F-Score: 0.4595
Recall: 0.3469, Precision: 0.68
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1095
           1       0.35      0.15      0.21        87

    accuracy                           0.92      1182
   macro avg       0.64      0.56      0.58      1182
weighted avg       0.89      0.92      0.90      1182

